In [29]:
import pandas as pd
import geopandas as gpd
import pathlib
from pathlib import Path
from tqdm import tqdm
import gzip
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
import xml.etree.ElementTree as ET
# to read the excel
from openpyxl import load_workbook
from openpyxl import Workbook
import datetime
import os
import re
import numpy as np
from collections import OrderedDict
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from plotnine import *
from plotnine.data import *
# set the working directory
BASE_DIR = Path.cwd()

In [30]:
#Sample information

sample_pct = 0.0088 # sample value in percentage
scale_factor = 100/(100*sample_pct)

In [31]:
# get the first and the last iteration name
def get_iteration_count(simulation_run):
    rootPath = BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS")
    list_iter=[]
    for name in [entry for entry in os.listdir(rootPath) if os.path.isdir(os.path.join(rootPath, entry))]:
        list_iter.append(int(name.rsplit("it.")[1]))
    return max((list_iter)),min((list_iter))
# setup the BEAM model output folder name here
simulation_run = "calibration-trial-20__gmu"
last_iter,first_iter = get_iteration_count(simulation_run)

# get the path of the simulation run and also get its creation date
def get_foldercreation_inf(folder_name):
    fname = pathlib.Path(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(last_iter)), (str(last_iter)+".linkstats.csv.gz")))
    assert fname.exists(), f'No such file: {fname}'  # check that the file exists
    ctime = datetime.datetime.fromtimestamp(fname.stat().st_ctime)
    return ctime.strftime("%Y-%m-%d")
# get its creation date
date_time = get_foldercreation_inf(simulation_run)

In [32]:
# get VMT, VHT and ratio from the last iteration of the linkstats file
class linkstats_to_df(object):
    def __init__(self,df):
        self.dataframe = df
        self.dataframe = self.dataframe.reset_index(drop=True)
        self.dataframe["volume"] = self.dataframe["volume"].fillna(0)
        self.dataframe["length"] = self.dataframe["length"].fillna(0)
        self.dataframe["vmt"] = self.dataframe["length"]*0.00062137*self.dataframe["volume"]*scale_factor
        self.VMT_sum = self.dataframe["vmt"].sum()
        # add congested speed (mph)
        self.dataframe["congested_speed_mph"] = (self.dataframe["length"]/self.dataframe["traveltime"])*2.23694
        # add free-flow speed (mph)
        self.dataframe["freeflow_speed_mph"] = self.dataframe["freespeed"]*2.23694
        # add vehicle hours of delay (free flow speed - congested speed)* volume
        self.dataframe["VHD"] = ((self.dataframe["freeflow_speed_mph"]-self.dataframe["congested_speed_mph"])*(self.dataframe["volume"])*20)/(60*60)
        self.VHD_sum = self.dataframe["VHD"].sum()
        # calculate VHT
        self.dataframe["VHT"] = self.dataframe["volume"]*(self.dataframe["traveltime"]/(60*60))*scale_factor
        self.VHT_sum = self.dataframe["VHT"].sum()
        # calculate VMT VHT Ratio
        self.VMT_VHT_ratio = self.VMT_sum/self.VHT_sum

dfBayAreaBeam = linkstats_to_df(pd.read_csv((BASE_DIR.parent.joinpath("runs",simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".linkstats.csv.gz")))))
# print("LinkStatistics")
# print("VMT: {}, VHT: {}, ratio:{}".format(dfBayAreaBeam.VMT_sum,dfBayAreaBeam.VHT_sum,dfBayAreaBeam.VMT_VHT_ratio))

In [33]:
### now read the link stat from the SF County
gdSFCountyNetwork = gpd.read_file((BASE_DIR.parent.joinpath("runs", simulation_run,"Network","SFCounty_EPSG4326.geojson")),driver='GeoJSON')
gdfSFBoundary = gpd.read_file((BASE_DIR.parent.joinpath("runs", simulation_run,"Network","SFCounty_boundary_EPSG4326.geojson")),driver='GeoJSON')

df = gdSFCountyNetwork.loc[~gdSFCountyNetwork["Information"].isin(['San Mateo', 'Golden Gate Bridge', 'SFOak1', 'SFOak2'])]
# # convert it to dictinary "Length" & "Link"
network_links = dict(zip(df["ID"], df["LENGTH"]))

dfBayArea = pd.read_csv(BASE_DIR.parent.joinpath((BASE_DIR.parent.joinpath("runs",simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".linkstats.csv.gz")))), compression="gzip", low_memory=True)
dfBayArea.reset_index(drop=True,inplace=True)
dfBayArea["link"]=dfBayArea["link"].astype(str)
dfSFCounty=dfBayArea[dfBayArea["link"].isin(network_links.keys())]
dfSFCountyBeam = linkstats_to_df(dfSFCounty)
# print("LinkStatistics")
# print("VMT: {}, VHT: {}, ratio:{}".format(dfSFCountyBeam.VMT_sum,dfSFCountyBeam.VHT_sum,dfSFCountyBeam.VMT_VHT_ratio))

In [34]:
class detailed_events(object):
    def __init__(self,folder_path):
        self.dfEvents = pd.read_csv(folder_path,low_memory=False)
        self.dfEvents["Travel_Time"] = self.dfEvents["arrivalTime"] - self.dfEvents["departureTime"]
        self.TT_PTBoardings = {}
        PersonEnterVeh = self.dfEvents.loc[(self.dfEvents["type"]=="PersonEntersVehicle") &
                                           (~self.dfEvents["vehicle"].str.contains("rideHailVehicle",na=False)) &
                                           (~self.dfEvents["person"].str.contains("TransitDriverAgent",na=False))].copy()
        # create dictionary: vehicle:mode
        veh_mode_df = self.dfEvents.loc[(self.dfEvents["type"]=="PathTraversal") &(self.dfEvents["vehicle"].str.contains("SF|BA", na=False))]
        veh_mode = dict(zip(veh_mode_df.vehicle, veh_mode_df["mode"]))
        # map them to get number of boardings by PT mode
        PersonEnterVeh["mode_veh"] = PersonEnterVeh["vehicle"].map(veh_mode,na_action='ignore')
        # get counts
        self.veh_counts = PersonEnterVeh[~PersonEnterVeh["mode_veh"].isna()].mode_veh.value_counts()
        self.bus_boarding = self.veh_counts.bus
        self.TT_PTBoardings["bus_boarding"] = self.veh_counts.bus
        self.tram_boarding = self.veh_counts.tram
        self.TT_PTBoardings["tram_boarding"] = self.veh_counts.tram
        self.rail_boarding = self.veh_counts.subway
        self.TT_PTBoardings["rail_boarding"] = self.veh_counts.subway
        self.cable_boarding = self.veh_counts.cable_car
        self.TT_PTBoardings["cable_boarding"] = self.veh_counts.cable_car

    def get_ii_modeshare(self):
        dfEvents = self.dfEvents[~self.dfEvents["person"].str.contains("ix",na=False) & ~self.dfEvents["person"].str.contains("rideHailAgent",na=False) & ~self.dfEvents["person"].str.contains("TransitDriverAgent",na=False) & (~self.dfEvents["person"].isna())]
        return dfEvents.loc[(dfEvents["type"]=="TripArrivalEvent"),"mode"].value_counts(normalize=True)

    def get_TT_modeshare(self):
        dfEvents = self.dfEvents[~self.dfEvents["person"].str.contains("rideHailAgent",na=False) & ~self.dfEvents["person"].str.contains("TransitDriverAgent",na=False) & (~self.dfEvents["person"].isna())]
        return dfEvents.loc[(dfEvents["type"]=="TripArrivalEvent"),"mode"].value_counts(normalize=True)

    def get_ii_PTBoardings(self):
        dfEvents_ii = self.dfEvents.copy()
        PTBoardings = {}
        dfEvents_ii = dfEvents_ii.loc[~dfEvents_ii["person"].str.startswith("ix", na=False)]
        PersonEnterVeh = dfEvents_ii.loc[(dfEvents_ii["type"]=="PersonEntersVehicle") &
                                         (~dfEvents_ii["vehicle"].str.contains("rideHailVehicle",na=False)) &
                                         (~dfEvents_ii["person"].str.contains("TransitDriverAgent",na=False))]
        pt_ridreship = len(PersonEnterVeh[PersonEnterVeh["vehicle"].str.contains("SF|BA", na=False)])
        # create dictionary: vehicle:mode
        veh_mode_df = dfEvents_ii.loc[(dfEvents_ii["type"]=="PathTraversal") & (dfEvents_ii["vehicle"].str.contains("SF|BA", na=False))]
        veh_mode = dict(zip(veh_mode_df.vehicle, veh_mode_df["mode"]))
        # map them to get number of boardings by PT mode
        PersonEnterVeh["mode_veh"] = PersonEnterVeh["vehicle"].map(veh_mode,na_action='ignore')
        # get counts
        veh_counts = PersonEnterVeh[~PersonEnterVeh["mode_veh"].isna()].mode_veh.value_counts()
        PTBoardings["bus_boarding"] = veh_counts.bus
        PTBoardings["tram_boarding"] =veh_counts.tram
        PTBoardings["rail_boarding"] =veh_counts.subway
        PTBoardings["cable_boarding"] = veh_counts.cable_car
        return PTBoardings

    def get_agents_count(self):
        agent_counts = {}
        agent_counts["ix_agents"]  = self.dfEvents.loc[self.dfEvents["person"].str.startswith("ix", na=False),"person"].nunique()
        agent_counts["ii_agents"]  = self.dfEvents.loc[(~ self.dfEvents["person"].str.startswith("ix", na=False)) & (~self.dfEvents["person"].str.startswith("rideHailAgent",na=False)) & (~self.dfEvents["person"].str.startswith("TransitDriverAgent",na=False)),"person"].nunique()
        agent_counts["ridehail_agent"] = self.dfEvents.loc[self.dfEvents["person"].str.startswith("rideHailAgent", na=False),"person"].nunique()
        agent_counts["total_agent"] = sum(agent_counts.values())
        return agent_counts

    def get_TT_unlinked_trips(self):
        dfEvents = self.dfEvents[~self.dfEvents["person"].str.contains("rideHailAgent",na=False) & ~self.dfEvents["person"].str.contains("TransitDriverAgent",na=False) & (~self.dfEvents["person"].isna())]
        return dfEvents.loc[(dfEvents["type"]=="TripArrivalEvent"),"mode"].value_counts()

    def get_ii_unlinked_trips(self):
        dfEvents = self.dfEvents[~self.dfEvents["person"].str.contains("ix",na=False) & ~self.dfEvents["person"].str.contains("rideHailAgent",na=False) & ~self.dfEvents["person"].str.contains("TransitDriverAgent",na=False) & (~self.dfEvents["person"].isna())]
        return dfEvents.loc[(dfEvents["type"]=="TripArrivalEvent"),"mode"].value_counts()

    def get_TT_tourpurpose_stats(self):
        dfEvents = self.dfEvents.copy()
        dfTour = dfEvents.loc[(~dfEvents["person"].str.contains("rideHailAgent",na=False) &
                               ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) &
                               (~dfEvents["person"].isna())),:].dropna(subset = ["vehicleOwnership"])
        return dfTour\
            .groupby(["tourPurpose","vehicleOwnership","mode"]).size().reset_index().rename(columns={0:"count"})\
            .groupby("tourPurpose")\
            .apply(lambda vehOwntype: dict(vehOwntype.groupby("vehicleOwnership")
                                           .apply(lambda mode_used: dict(zip(mode_used["mode"],mode_used["count"])))))

    def get_ii_tourpurpose_stats(self):
        dfEvents = self.dfEvents.copy()
        dfTour = dfEvents.loc[(~dfEvents["person"].str.contains("ix",na=False) &
                               ~dfEvents["person"].str.contains("rideHailAgent",na=False) &
                               ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) &
                               (~dfEvents["person"].isna())),:].dropna(subset = ["vehicleOwnership"])
        dfTour=dfTour[dfTour["type"]=="TripArrivalEvent"]
        tour_purpose_dict = dfTour.groupby(["tourPurpose", "vehicleOwnership", "mode"]).size().reset_index().rename(columns={0: "count"})\
            .groupby("tourPurpose")\
            .apply(lambda vehOwntype: dict(vehOwntype.groupby("vehicleOwnership")
                                           .apply(lambda mode_used: dict(zip(mode_used["mode"], mode_used["count"]))))).copy()
        for key,value in tour_purpose_dict.iteritems():
            if isinstance(tour_purpose_dict.get(key),dict):
                for auto_ownership,modes in tour_purpose_dict.get(key).items():
                    mode_keys = ["car", "walk_transit", "drive_transit", "bike_transit", "ride_hail_transit", "walk", "bike", "ride_hail_pooled", "ride_hail", "hov2_teleportation", "hov2","hov3_teleportation", "hov3"]
                    if isinstance(modes,dict):
                        for mode,count in modes.items():
                            if mode in mode_keys:
                                mode_keys.remove(mode)
                        for mode in mode_keys:
                            tour_purpose_dict[key][auto_ownership][mode]=0

        return tour_purpose_dict

    def get_ii_auto_ratio_status(self):
        dfEvents = self.dfEvents.copy()
        dfTour = dfEvents.loc[(~dfEvents["person"].str.contains("ix",na=False) &
                               ~dfEvents["person"].str.contains("rideHailAgent",na=False) &
                               ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) &
                               (~dfEvents["person"].isna())),:].dropna(subset = ["vehicleOwnership"])
        return dfTour["vehicleOwnership"].value_counts(normalize=True)

    def get_TT_auto_ratio_status(self):
        dfEvents = self.dfEvents.copy()
        dfTour = dfEvents.loc[(~dfEvents["person"].str.contains("rideHailAgent",na=False) &
                               ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) &
                               (~dfEvents["person"].isna())),:].dropna(subset = ["vehicleOwnership"])
        return dfTour["vehicleOwnership"].value_counts(normalize=True)

    def get_TT_activity_type(self):
        dfEvents = self.dfEvents.copy()
        dfActivity = dfEvents.loc[(~dfEvents["person"].str.contains("rideHailAgent",na=False) &
                                   ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) &
                                   (~dfEvents["person"].isna())),:].dropna(subset = ["tourPurpose"])
        return dfActivity["tourPurpose"].value_counts(normalize=True)

    def get_ii_activity_type(self):
        dfEvents = self.dfEvents.copy()
        dfActivity = dfEvents.loc[(~dfEvents["person"].str.contains("ix",na=False) &
                                   ~dfEvents["person"].str.contains("rideHailAgent",na=False) &
                                   ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) &
                                   (~dfEvents["person"].isna())),:].dropna(subset = ["tourPurpose"])
        return dfActivity["tourPurpose"].value_counts(normalize=True)

    def get_ii_transfers(self):
        dfEvents = self.dfEvents.copy()
        dfEvents = dfEvents[~dfEvents["person"].str.contains("ix",na=False) & ~dfEvents["person"].str.contains("rideHailAgent",na=False) & ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) & (~dfEvents["person"].isna())].copy()
        # dfEvents = dfEvents.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:].copy()

        dfEvents["numberofTransfers"]=dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[-22:].str.split(")",expand=True)[0].str.split(",",expand=True)[1]
        dfEvents["trip_cost_utility"]=dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[-34:-25].str.split(",",expand=True)[1]
        dfEvents["walkTransitDistances"] = dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[1:-37].str.split(",",expand=True)[1].str.split(":",expand=True)[0]
        dfEvents["walkTransitDistances"] = pd.to_numeric(dfEvents["walkTransitDistances"], errors='coerce').replace(np.nan,0,regex=True)
        dfEvents["trip_cost_utility"] = pd.to_numeric(dfEvents["trip_cost_utility"], errors='coerce').replace(np.nan,0,regex=True)
        dfEvents["numberofTransfers"] = pd.to_numeric(dfEvents["numberofTransfers"], errors='coerce').replace(np.nan,0,regex=True).round(0).astype(int)

        dData = dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]
        dData.loc[:,"walkTransitDistances_mil"] = dData["walkTransitDistances"]*0.000621371

        def categorized_distance(cell_value):
            if cell_value>1:
                walkTransitDistances_cat=">1mil"
            elif ( cell_value<=1.0) & (cell_value>0.75):
                walkTransitDistances_cat=">0.75mil & <=1mil"
            elif ( cell_value<=0.75) & (cell_value>0.5):
                walkTransitDistances_cat=">0.5mil & <=0.75mil"
            elif (cell_value<=0.50) & (cell_value>0.25):
                walkTransitDistances_cat=">0.25mil & <=0.5mil"
            elif (cell_value<=0.25):
                walkTransitDistances_cat="<=0.25mil"
            return walkTransitDistances_cat

        dData.loc[:,"walkTransitDistances_cat"] = dData["walkTransitDistances_mil"].apply(lambda x:categorized_distance(x))

        def categorized_transfer(cell_value):
            if cell_value==0:
                categorized_transfer_cat="0"
            elif ( cell_value<=1.0) & (cell_value>0.0):
                categorized_transfer_cat="1"
            elif (cell_value<=2) & (cell_value>1):
                categorized_transfer_cat="2"
            elif (cell_value>2):
                categorized_transfer_cat="3+"
            return categorized_transfer_cat

        dData.loc[:,"numberofTransfers_cat"] = dData["numberofTransfers"].apply(lambda x:categorized_transfer(x))

        transfer_dict = (dData.groupby(["tourPurpose", "numberofTransfers_cat","walkTransitDistances_cat"]).size().reset_index().rename(columns={0: "count"}).groupby("tourPurpose").apply(lambda vehOwntype: dict(vehOwntype.groupby("numberofTransfers_cat").apply(lambda mode_used: dict(zip(mode_used["walkTransitDistances_cat"], mode_used["count"]))))).copy())

        for key,values in transfer_dict.iteritems():
            # print(key)
            cat = ["0","1","2","3+"]
            for transfer_cat,dist in transfer_dict.get(key).items():
                # print("\t",transfer_cat)
                cat.remove(transfer_cat)
            for c in cat:
                # print(c)
                transfer_dict[key][c]={"<=0.25mil":0,">0.25mil & <=0.5mil":0,">0.5mil & <=0.75mil":0,">0.75mil & <=1mil":0,">1mil":0 }

        for key,value in transfer_dict.iteritems():
            if isinstance(transfer_dict.get(key),dict):
                for transfer,walk_distance in transfer_dict.get(key).items():
                    walk_distance_keys = ["<=0.25mil",">0.25mil & <=0.5mil",">0.5mil & <=0.75mil",">0.75mil & <=1mil",">1mil"]
                    if isinstance(walk_distance,dict):
                        for cat,count in walk_distance.items():
                            if cat in walk_distance_keys:
                                walk_distance_keys.remove(cat)
                        for cat in walk_distance_keys:
                            transfer_dict[key][transfer][cat]=0

        return transfer_dict

    def get_ii_transfers_firstmile_summary(self):
        dfEvents = self.dfEvents.copy()
        dfEvents = dfEvents[~dfEvents["person"].str.contains("ix",na=False) & ~dfEvents["person"].str.contains("rideHailAgent",na=False) & ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) & (~dfEvents["person"].isna())].copy()
        # dfEvents = dfEvents.loc[(df["type"]=="ModeChoice") & (df["mode"]=="walk_transit"),:].copy()

        dfEvents["numberofTransfers"]=dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[-22:].str.split(")",expand=True)[0].str.split(",",expand=True)[1]
        dfEvents["trip_cost_utility"]=dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[-34:-25].str.split(",",expand=True)[1]
        dfEvents["walkTransitDistances"] = dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[1:-37].str.split(",",expand=True)[1].str.split(":",expand=True)[0]

        dfEvents["walkTransitDistances"] = pd.to_numeric(dfEvents["walkTransitDistances"], errors='coerce').replace(np.nan,0,regex=True)
        dfEvents["trip_cost_utility"] = pd.to_numeric(dfEvents["trip_cost_utility"], errors='coerce').replace(np.nan,0,regex=True)
        dfEvents["numberofTransfers"] = pd.to_numeric(dfEvents["numberofTransfers"], errors='coerce').replace(np.nan,0,regex=True).round(0).astype(int)

        dData = dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]
        # dData.loc[:,"walkTransitDistances_mil"] = dData.loc[:,"walkTransitDistances"]*0.000621371
        dData.loc[:,"walkTransitDistances_mil"] = dData["walkTransitDistances"]*0.000621371

        def categorized_distance(cell_value):
            if cell_value>1:
                walkTransitDistances_cat=">1mil"
            elif ( cell_value<=1.0) & (cell_value>0.75):
                walkTransitDistances_cat=">0.75mil & <=1mil"
            elif ( cell_value<=0.75) & (cell_value>0.5):
                walkTransitDistances_cat=">0.5mil & <=0.75mil"
            elif (cell_value<=0.50) & (cell_value>0.25):
                walkTransitDistances_cat=">0.25mil & <=0.5mil"
            elif (cell_value<=0.25):
                walkTransitDistances_cat="<=0.25mil"
            return walkTransitDistances_cat

        dData.loc[:,"walkTransitDistances_cat"] = dData["walkTransitDistances_mil"].apply(lambda x:categorized_distance(x))

        def categorized_transfer(cell_value):
            if cell_value==0:
                categorized_transfer_cat="0"
            elif ( cell_value<=1.0) & (cell_value>0.0):
                categorized_transfer_cat="1"
            elif (cell_value<=2) & (cell_value>1):
                categorized_transfer_cat="2"
            elif (cell_value>2):
                categorized_transfer_cat="3+"
            return categorized_transfer_cat

        dData.loc[:,"numberofTransfers_cat"] = dData["numberofTransfers"].apply(lambda x:categorized_transfer(x))
        transfer_dict = (dData.groupby(["walkTransitDistances_cat","numberofTransfers_cat"]).size().reset_index().rename(columns={0: "count"}).groupby("walkTransitDistances_cat").apply(lambda mode_used: dict(zip(mode_used["numberofTransfers_cat"], mode_used["count"])))).copy()
        for key,values in transfer_dict.iteritems():
            cat = ["0","1","2","3+"]
            if isinstance(values,dict):
                for k,v in transfer_dict.get(key).items():
                    if k in cat:
                        cat.remove(k)
                for c in cat:
                    transfer_dict[key][c]=0

        return transfer_dict

    def get_ii_nTransfers(self):
        dfEvents = self.dfEvents.copy()
        dfEvents = dfEvents[~dfEvents["person"].str.contains("ix",na=False) & ~dfEvents["person"].str.contains("rideHailAgent",na=False) & ~dfEvents["person"].str.contains("TransitDriverAgent",na=False) & (~dfEvents["person"].isna())].copy()
        dfEvents["numberofTransfers"]=dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[-22:].str.split(")",expand=True)[0].str.split(",",expand=True)[1]
        dfEvents["trip_cost_utility"]=dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[-34:-25].str.split(",",expand=True)[1]
        dfEvents["walkTransitDistances"] = dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]["tracingData"].str[1:-37].str.split(",",expand=True)[1].str.split(":",expand=True)[0]
        dfEvents["walkTransitDistances"] = pd.to_numeric(dfEvents["walkTransitDistances"], errors='coerce').replace(np.nan,0,regex=True)
        dfEvents["trip_cost_utility"] = pd.to_numeric(dfEvents["trip_cost_utility"], errors='coerce').replace(np.nan,0,regex=True)
        dfEvents["numberofTransfers"] = pd.to_numeric(dfEvents["numberofTransfers"], errors='coerce').replace(np.nan,0,regex=True).round(0).astype(int)

        dData = dfEvents.loc[(dfEvents["type"]=="ModeChoice") & (dfEvents["mode"]=="walk_transit"),:]
        dData.loc[:,"walkTransitDistances_mil"] = dData["walkTransitDistances"]*0.000621371
        return dData.copy()

In [45]:
dfBayAreaBeamEvents = detailed_events(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".events.csv.gz")))
dfBayAreaZerothEvents = detailed_events(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(first_iter)),(str(first_iter)+".events.csv.gz")))

In [50]:
dfLast = dfBayAreaBeamEvents.dfEvents
dfFirst = dfBayAreaZerothEvents.dfEvents

In [49]:
dfFirst

,currentTourMode,driver,vehicle,numPassengers,vehicleType,primaryFuel,links,riders,length,seatingCapacity,...,tracingData,availableAlternatives,routerAvailableAlternatives,location,personalVehicleAvailable,link,facility,score,reason,Travel_Time
0,NaN,NaN,BA:23DCM21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,SF:7634132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,SF:7600841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,SF:7580830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,BA:186OAK10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892918,NaN,ix-1298109-1,auto-1298109-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0,NaN,NaN
892919,NaN,TransitDriverAgent-SF:7633338,SF:7633338,0.0,BUS-DEFAULT,1.055335e+07,"18930,96456,1058,18920,50778,52372",NaN,526.404,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0
892920,car,ix-1298109-1,auto-1298109-1,0.0,Car,0.000000e+00,58085,NaN,0.000,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
892921,NaN,TransitDriverAgent-SF:7600910,SF:7600910,0.0,BUS-DEFAULT,4.557793e+06,"84682,51812,84686,19544,19659",NaN,227.344,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0


In [57]:
import random

# get the number of transfers pic
# df = dfBayAreaBeamEvents.dfEvents
df = dfBayAreaBeamEvents.get_ii_nTransfers()

person_list = df.loc[df["numberofTransfers"]>=2, "person"].to_list()
personid = random.choice(person_list)
print(f"Agent ID: {personid}")
# df.loc[df["numberofTransfers"]>1,["person","numberofTransfers"]].to_csv(BASE_DIR.parent.joinpath("exported", f"FinalIteration-2andup_transfer_agents.csv"))
# dfLast = dfBayAreaBeamEvents.dfEvents

dfLast.loc[dfLast["person"]==personid].to_csv(BASE_DIR.parent.joinpath("exported", f"FinalIteration-agent-{personid}.csv"))

# dfFirst = dfBayAreaZerothEvents.dfEvents
dfFirst.loc[dfFirst["person"]==personid].to_csv(BASE_DIR.parent.joinpath("exported", f"ZeroIteration-agent-{personid}.csv"))

Agent ID: 36470-3


C:\Users\Goyal\AppData\Local\Temp\ipykernel_18028\3093927051.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [59]:
df = pd.read_csv((BASE_DIR.parent.joinpath("runs",simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".physSimEvents.csv.gz"))))

In [62]:
df.loc[df["person"]=="SF:7676689"]

,person,link,legMode,time,type,vehicle,mode,currentTourMode,income,expectedMaximumUtility,...,length,tourIndex,tourPurpose,facility,actType,activityIndex,locationX,locationY,networkMode,relativePosition
93334,SF:7676689,97034,NaN,22471.000000,actend,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,DummyActivity,NaN,NaN,NaN,NaN,NaN
93335,SF:7676689,97034,car,22471.000100,departure,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93337,SF:7676689,97034,NaN,22471.001000,vehicle enters traffic,SF:7676689,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
99628,SF:7676689,13378,NaN,22881.783609,vehicle leaves traffic,SF:7676689,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,car,1.0
99629,SF:7676689,13378,car,22881.783709,arrival,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99630,SF:7676689,13378,NaN,22881.783809,actstart,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,DummyActivity,NaN,NaN,NaN,NaN,NaN
99631,SF:7676689,13378,NaN,22881.784609,actend,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,DummyActivity,NaN,NaN,NaN,NaN,NaN
99632,SF:7676689,13378,car,22881.784709,departure,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99633,SF:7676689,13378,NaN,22881.785609,vehicle enters traffic,SF:7676689,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
100217,SF:7676689,41449,NaN,22923.905961,vehicle leaves traffic,SF:7676689,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,car,1.0


In [64]:
df = pd.read_csv((BASE_DIR.parent.joinpath("runs",simulation_run,"network.csv")))

In [69]:
df

,linkId,linkLength,linkFreeSpeed,linkCapacity,numberOfLanes,linkModes,attributeOrigId,attributeOrigType,fromNodeId,toNodeId,fromLocationX,fromLocationY,toLocationX,toLocationY
0,0,244.148,26.82,7600.0,4.0,car;walk;bike,1.0,freeway,0,1,553350.000950,4.173346e+06,553320.943482,4.173588e+06
1,1,244.148,26.82,7600.0,4.0,walk;bike,1.0,freeway,1,0,553320.943482,4.173588e+06,553350.000950,4.173346e+06
2,10,330.865,8.69,950.0,1.0,car;walk;bike,6.0,maj_arterial,8,10,546770.002863,4.173201e+06,546668.684147,4.173516e+06
3,100,224.255,7.45,350.0,1.0,car;walk;bike,51.0,local,63,64,554880.077637,4.176043e+06,554685.532201,4.176155e+06
4,1000,146.746,8.69,1600.0,2.0,car;walk;bike,501.0,minor_arterial,354,294,553925.906528,4.177444e+06,554047.315707,4.177362e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97195,9995,131.358,7.45,650.0,1.0,walk;bike,4998.0,collector,2360,2453,549963.310450,4.176377e+06,550084.568256,4.176428e+06
97196,9996,113.027,7.45,300.0,1.0,car;walk;bike,4999.0,local,2536,2453,549974.874770,4.176457e+06,550084.568256,4.176428e+06
97197,9997,113.027,7.45,300.0,1.0,walk;bike,4999.0,local,2453,2536,550084.568256,4.176428e+06,549974.874770,4.176457e+06
97198,9998,100.913,7.45,300.0,1.0,car;walk;bike,5000.0,local,2536,2538,549974.874770,4.176457e+06,549876.937661,4.176482e+06


In [68]:
df.loc[df["linkId"]==40190]

,linkId,linkLength,linkFreeSpeed,linkCapacity,numberOfLanes,linkModes,attributeOrigId,attributeOrigType,fromNodeId,toNodeId,fromLocationX,fromLocationY,toLocationX,toLocationY
33548,40190,193.688,8.69,1600.0,2.0,car;walk;bike,20096.0,minor_arterial,10087,10096,545413.285516,4.180914e+06,545401.160019,4.181107e+06
